In [ ]:
!pip install -q transformers datasets timm safetensors

print("danish")

In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import ViTFeatureExtractor, ViTForImageClassification
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import requests
from io import BytesIO
from safetensors.torch import save_file
from IPython.display import FileLink

print("danish")

In [4]:

train_dir = "train"
valid_dir = "valid"

In [5]:
# 5. Load ViT Feature Extractor
from transformers import ViTImageProcessor

feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

print("Danish")


Danish


In [6]:
classes = sorted(os.listdir(train_dir))
print(f"Detected classes ({len(classes)}): {classes}")



Detected classes (40): ['.DS_Store', 'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Random___image', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spid

In [8]:
class PlantDataset(Dataset):
    def __init__(self, root_dir, transform=None, class_names=None):
        """
        :param root_dir: Path to the folder (train, valid, or test)
        :param transform: Transformations to be applied
        :param class_names: List of class names (subfolders)
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = class_names
        
        # Gather all images and labels
        for label, class_name in enumerate(self.class_names):
            class_path = os.path.join(self.root_dir, class_name)
            if not os.path.isdir(class_path):
                # If the folder doesn't exist, skip
                continue
            
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, torch.tensor(label)

print("Danish")


Danish


In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

# Create Dataset objects
train_dataset = PlantDataset(root_dir=train_dir, transform=transform, class_names=classes)
valid_dataset = PlantDataset(root_dir=valid_dir, transform=transform, class_names=classes)

print("Danish")


Danish


In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)


print("Danish")

Danish


In [12]:
print(f"Train samples: {len(train_dataset)}")
print(f"Valid samples: {len(valid_dataset)}")

Train samples: 76764
Valid samples: 21062


In [13]:
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(classes),
    id2label={str(i): classes[i] for i in range(len(classes))},
    label2id={classes[i]: i for i in range(len(classes))}
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Danish")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Danish


In [14]:
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss()

print("danish")

danish


In [ ]:
def train_model(model, train_loader, valid_loader, epochs=4):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        # Training loop
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]", leave=True)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            train_bar.set_postfix({"loss": f"{loss.item():.4f}"})
        
        train_loss = total_loss / len(train_loader)
        train_acc = 100 * correct / total
        
        # Validation loop
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss_sum = 0
        
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).logits
                val_loss = criterion(outputs, labels)
                val_loss_sum += val_loss.item()
                _, predicted = torch.max(outputs, dim=1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
        
        val_loss_avg = val_loss_sum / len(valid_loader)
        val_acc = 100 * val_correct / val_total
        
        print(f"Epoch [{epoch+1}/{epochs}]")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Valid Loss: {val_loss_avg:.4f} | Valid Acc: {val_acc:.2f}%\n")
        
        # Save checkpoint each epoch (if desired)
        torch.save(model.state_dict(), f"vit_epoch{epoch+1}.pt")

print("Danish")

In [ ]:
import os
from PIL import Image

dataset_dir = "train"
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        try:
            Image.open(img_path).convert("RGB")
        except Exception as e:
            print("Corrupted file:", img_path, e)

            
print("danish")

In [ ]:
train_model(model, train_loader, valid_loader, epochs=4)


print("Danish")

Epoch 1/4 [Training]:   0%|                                                                     | 0/2399 [00:00<?, ?it/s]

In [ ]:
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total

In [ ]:
sample_url = "https://projectbluearchive.blob.core.windows.net/media/Default/Potatoes/Disease%20and%20defects/Alternaria%20Leaf%20resized.jpg"
response = requests.get(sample_url)
sample_image = Image.open(BytesIO(response.content)).convert("RGB")

# Preprocess the image (same as training)
inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])
input_image = inference_transform(sample_image).unsqueeze(0).to(device)

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(input_image).logits
pred_idx = torch.argmax(logits, dim=1).item()

print(f"Predicted class index: {pred_idx}")
print(f"Predicted label: {classes[pred_idx]}")

In [ ]:
model_path = "plant_disease_model.safetensors"
save_file(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Provide a download link (for environments like JupyterLab or Kaggle)
FileLink(model_path)